In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.layers import Dense
import sys
from keras.callbacks import EarlyStopping
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import Input, GRU, TimeDistributed, Dense, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, GRU, TimeDistributed, Dense, Dropout, Reshape, Attention, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, GRU, TimeDistributed, Dense, Dropout, Reshape, Flatten, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.models import Model

2024-07-12 00:05:04.830247: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 00:05:04.898167: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 00:05:04.969482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 00:05:05.022072: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 00:05:05.038327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 00:05:05.148289: I tensorflow/core/platform/cpu_feature_gu

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
final_arousal_df = pd.read_csv('Extracted_Features/Final_Arousal_Features.csv')
final_valence_df = pd.read_csv('Extracted_Features/Final_Valence_Features.csv')

In [4]:
final_valence_df.drop(["Arousal","Gender", "participant", "video", "Video_Name", "Age"], inplace=True, axis=1)
final_arousal_df.drop(["Valence", "Gender", "participant", "video", "Video_Name", "Age"], inplace=True, axis=1)

In [16]:
# Function to convert string representation of list to a list of floats
def str_to_float_list(s):
    return list(map(float, s.strip('[]').split()))

In [17]:
columns_to_convert = []
for col in final_valence_df.columns:
    if final_valence_df[col].dtype == 'object' and col != 'Target_Emotion':
        columns_to_convert.append(col)

In [18]:
for col in columns_to_convert:
    final_valence_df[col] = final_valence_df[col].apply(str_to_float_list)

In [23]:
X = final_valence_df.drop(columns=['Valence', 'Target_Emotion'])  # Drop the target columns
y = final_valence_df['Valence']  # Target column

In [24]:
def flatten_features(X):
    return np.array(X.apply(lambda row: np.hstack(row.values), axis=1).tolist())

X_flattened = flatten_features(X)

## Random Forest Regressor

In [25]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flattened, y, test_size=0.2, random_state=42)

# Model training with Random Forest Regressor
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [26]:
# Predictions and evaluation
y_pred = model.predict(X_test)

In [27]:
mse = mean_squared_error(y_test, y_pred)

In [28]:
r2 = r2_score(y_test, y_pred)

In [29]:
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 1.8003421686746985
R^2 Score: -0.03752360716078251


## Using Conv1D 

In [30]:
# Splitting the features (X) and target (y)
X = final_valence_df.drop(columns=['Valence', 'Target_Emotion'])  # Drop the target columns
y = final_valence_df['Valence']  # Target column

In [31]:
# Convert features to a 3D array
def convert_to_3d_array(df, columns):
    return np.stack(df[columns].apply(lambda row: np.stack(row.values), axis=1).values)

X_3d = convert_to_3d_array(final_valence_df, columns_to_convert)

ValueError: all input arrays must have the same shape